In [71]:
import os
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn.functional as F
# torch.multiprocessing.set_start_method('spawn')
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch import optim

from transformers import CLIPProcessor, CLIPModel

In [72]:
device = "cuda" if torch.cuda.is_available() else "cpu"
CONTEXT_LENGTH = 77
SAMPLE_SIZE = None
TRAIN_PROP = 0.7
VAL_PROP = 0.2
BATCH_SIZE = 4
NUM_WORKERS = 0
EPOCHS = 4
LEARNING_RATE = 5e-5

# Params same as paper
BETAS = (0.9,0.98)
EPS=1e-6
WEIGHT_DECAY=0.2

In [73]:
summary_bert_path = "data/Summaries/Summary_Bert_65.csv"
summary_bert_path = "data/Summaries/export_summary_bert_with_ingredients.csv"
data_dir = "data"
images_dir = "Food Images/Food Images"

In [74]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# inputs = processor(
#     text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True
# )

# outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [63]:
class TextImg_Dataset(Dataset):
    def __init__(self, data_dir, images_dir, images, titles):
        self.data_dir = data_dir
        self.images_dir = images_dir
        self.images_paths = images
        self.titles = titles
        assert (len(titles)==len(images))

    def __len__(self):
        return len(self.images_paths)
    
    def __getitem__(self, index):
        txt = self.titles[index]
        img = self.images_paths[index]
        # print(type(text_tokens), text_tokens.shape)
        path = os.path.join(self.data_dir, self.images_dir, img)
        image = Image.open(path) # torch.Size([3, 224, 224])
        inputs = processor(text=txt, images=image, return_tensors="pt", padding=True,).to(device)
        print(inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['pixel_values'].shape)
        return inputs


In [64]:
summary_df = pd.read_csv(summary_bert_path)
liste_images = summary_df["Image_Name"].tolist()
liste_textes = summary_df["summary_with_ingredients"].tolist()
if SAMPLE_SIZE != None:
    liste_images=liste_images[:SAMPLE_SIZE]
    liste_textes=liste_textes[:SAMPLE_SIZE]
liste_images = [image + ".jpg" for image in liste_images]

In [65]:
liste_images[0], liste_textes[0]

('miso-butter-roast-chicken-acorn-squash-panzanella.jpg',
 'Roast chicken in a large cast-iron skillet until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes.Whole chicken')

In [66]:
assert(len(liste_textes) == len(liste_images))
length = len(liste_images)
print(length)
train_size = int(length*TRAIN_PROP)
val_size = int(length*VAL_PROP)

1221


In [68]:
train_dataset = TextImg_Dataset(data_dir, images_dir, liste_images[:train_size], liste_textes[:train_size])
val_dataset = TextImg_Dataset(data_dir, images_dir, liste_images[train_size:train_size+val_size], liste_textes[train_size:train_size+val_size])
test_dataset = TextImg_Dataset(data_dir, images_dir, liste_images[train_size+val_size:], liste_textes[train_size+val_size:])

In [69]:
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_dl = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_dl = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)

In [70]:
print(len(train_dl))
sample = next(iter(train_dl))

214


TypeError: _batch_encode_plus() got an unexpected keyword argument 'truncate'